In [26]:
from keras.models import Model
from keras.layers import Conv2D, Embedding, MaxPooling2D, Input, Dense, Dot
from keras.layers import Flatten, Reshape, Activation ,Concatenate, Reshape, Dropout
from keras.layers.normalization import BatchNormalization
from keras import backend as K
from keras.layers.wrappers import TimeDistributed, Bidirectional

from keras import optimizers
import tensorflow as tf
from keras.layers.recurrent import LSTM

import numpy as np

In [27]:
# input = (?, length, frequency) <- librosa mel-spectrum

In [52]:
class LSTM_attention():
    def __init__(self, num_freq = 128, len_div = 256, num_hidden=100):
        self.num_freq = num_freq
        self.len_div = len_div
        self.num_hidden = num_hidden
        
    def LSTM(self):

        self.inputs = Input(shape=(self.len_div, self.num_freq), name='input')

        self.lstm_for = LSTM(self.num_hidden, return_sequences=True, go_backwards=False, name='LSTM_for')(self.inputs)
        self.lstm_inv = LSTM(self.num_hidden, return_sequences=True, go_backwards=True, name='LSTM_inv')(self.inputs)
        self.conc = Concatenate(axis=2, name='concat')([self.lstm_for, self.lstm_inv])
        self.dens = Dense(self.num_freq, name='dense_for_inv')(self.conc)
        self.dot = Dot(axes=-1, name='dot')([self.inputs, self.dens])
#         self.argmax = K.argmax(self.dot, axis=-1)
        
        self.drop = Dropout(rate=0.05, name='drop')(self.dot)
        self.dens1 = Dense(80, name='dense')(self.drop)
        self.pred = Activation('softmax',name='pred')(self.dens1)

        adam = optimizers.Adam(lr=0.0001)
        model = Model(inputs=self.inputs, outputs=self.pred)
        model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
        return model
    
#     def Embed_LSTM(self):
#         adam = optimizers.Adam(lr=0.0001)
#         model = Model(inputs=self.inputs, outputs=self.lstm)
#         model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=[tf_lwlrap])
#         return model

In [53]:
lstm_attn = LSTM_attention()
model = lstm_attn.LSTM()
# model2 = lstm_attn.Embed_LSTM()

In [54]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 256, 128)     0                                            
__________________________________________________________________________________________________
LSTM_for (LSTM)                 (None, 256, 100)     91600       input[0][0]                      
__________________________________________________________________________________________________
LSTM_inv (LSTM)                 (None, 256, 100)     91600       input[0][0]                      
__________________________________________________________________________________________________
concat (Concatenate)            (None, 256, 200)     0           LSTM_for[0][0]                   
                                                                 LSTM_inv[0][0]                   
__________